### Import bank data from csv

In [1]:
import pandasai as pai
import pandas as pd
import ast
from dotenv import load_dotenv
import os 

load_dotenv()

pai_api_key = os.getenv("OPENAI_API_KEY")
if not pai_api_key:
    raise ValueError("PAI_API_KEY is not set")
else:
    print(f"OPENAI_API_KEY is set")

df = pd.read_csv("bank_data_23-25/barclays_072.csv")

# Sample DataFrame
df = pai.DataFrame(df)

pai.api_key.set(pai_api_key)


### extract currency and amount
df['currency'] = df['transactionAmount'].apply(lambda x: ast.literal_eval(x)['currency'])
df['amount'] = df['transactionAmount'].apply(lambda x: float(ast.literal_eval(x)['amount']))
df['amount'] = (df['amount'] * 100).astype(int)
df = df.drop('transactionAmount', axis=1)

## setting datetimeindex
df['bookingDate'] = pd.to_datetime(df['bookingDate'])
df.set_index('bookingDate', inplace=True)
df = df.drop(['valueDate', 'bookingDateTime', 'valueDateTime', 'internalTransactionId'], axis=1)

# df.chat("What are the total expenses for 2024?")


OPENAI_API_KEY is set


### XERO TIME!!

In [2]:
BankTransactions = { 
    "BankTransactions": [ { "Type": "SPEND", 
                           "Contact": { "ContactID": "00000220-0000-0000-0000-000000000000" }, 
                            "Lineitems": [ { "Description": "Foobar", "Quantity": 1, "UnitAmount": 20, "AccountCode": "400" } ], 
                            "BankAccount": { "Code": "088" } 
                            } ] 
}     


""" PUT https://api.xero.com/api.xro/2.0/BankTransactions
    EXAMPLE BODY GET REQUEST
{
  "BankTransactions": [
    {
      "Type": "SPEND",
      "Contact": {
        "ContactID": "ea791a0a-081c-4833-a4f1-3cccb323ec4a"  
      },
      "LineItems": [
        {
          "Description": "Foobar",
          "Quantity": 1.0,
          "UnitAmount": 20.0,
          "AccountCode": "433" 
        }
      ],
      "BankAccount": {
        "Code": "600" 
      }
    }
  ]
}
"""

' PUT https://api.xero.com/api.xro/2.0/BankTransactions\n    EXAMPLE BODY GET REQUEST\n{\n  "BankTransactions": [\n    {\n      "Type": "SPEND",\n      "Contact": {\n        "ContactID": "ea791a0a-081c-4833-a4f1-3cccb323ec4a"  \n      },\n      "LineItems": [\n        {\n          "Description": "Foobar",\n          "Quantity": 1.0,\n          "UnitAmount": 20.0,\n          "AccountCode": "433" \n        }\n      ],\n      "BankAccount": {\n        "Code": "600" \n      }\n    }\n  ]\n}\n'

### LLM Reconciliation Time

- Add new columns "coa_agent", "coa_reason", "coa_agent_confidence" for LLM


In [3]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import time
from typing import Dict, Tuple
import logging
import json


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

load_dotenv()

# Initialize the ChatGroq model
groq_model = ChatGroq(model_name="Llama-3.3-70b-Specdec")
openai_model = ChatOpenAI(model="gpt-4o")


class TransactionClassifier:
    def __init__(self):
        logger.info("Initializing TransactionClassifier")
        self.system_prompt = """
        You are a financial expert responsible for classifying transactions into appropriate chart of accounts.
        For each transaction, you must provide:
        1. The most specific appropriate chart of account code
        2. A brief explanation of why this classification was chosen. If confidence score is low, explain why.
        3. A confidence score between 0 and 1 (e.g., 0.95 for high confidence)

        You must respond with valid JSON in the following format only:
        {
            "code": "string",
            "reasoning": "string",
            "confidence": float
        }

        You will be given the chart of accounts, you must match the transaction against those accounts. 
        You must use the code field from the chart of accounts as the code field in the response.
        """
        self.last_request_time = 0
        self.rate_limit_delay = 2  # 2 seconds between requests (30 requests/minute)

    def _rate_limit(self):
        """Implement rate limiting"""
        current_time = time.time()
        time_since_last_request = current_time - self.last_request_time
        if time_since_last_request < self.rate_limit_delay:
            delay = self.rate_limit_delay - time_since_last_request
            logger.debug(f"Rate limiting: waiting {delay:.2f} seconds")
            time.sleep(delay)
        self.last_request_time = time.time() 

    def classify_transaction(self, transaction: Dict, chart_of_accounts: list) -> Tuple[str, str, float]:
        """Classify a single transaction using the LLM"""
        logger.info(f"Processing transaction: {transaction}")
        self._rate_limit()

        # Format the transaction details for the LLM
        transaction_prompt = f"""
        Please classify the following transaction:
        Transaction: {transaction}
        
        You must match the transaction against those accounts. 
        You must use the code field from the chart of accounts. 
        Chart of accounts: {chart_of_accounts}
        """

        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=transaction_prompt)
        ]

        try:
            logger.debug("Sending request to LLM")
            # Add more visible print statements
            print("\n=== Sending request to LLM ===")
            response = openai_model.invoke(messages)
            print("\n=== Raw LLM Response ===")
            print(response.content)
            logger.debug(f"Raw LLM response: {response.content}")

            # Try to parse the response as JSON
            try:
                # First, try direct JSON parsing
                result = json.loads(response.content)
                logger.info("Successfully parsed JSON response")
            except json.JSONDecodeError:
                # If direct parsing fails, try to extract JSON from the response
                logger.warning("Direct JSON parsing failed, attempting to extract JSON from response")
                # Look for JSON-like structure in the response
                import re
                json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
                if json_match:
                    result = json.loads(json_match.group())
                    logger.info("Successfully extracted and parsed JSON from response")
                else:
                    raise ValueError("No JSON structure found in response")

            # Validate the response structure
            required_keys = {'code', 'reasoning', 'confidence'}
            if not all(key in result for key in required_keys):
                missing_keys = required_keys - result.keys()
                raise ValueError(f"Missing required keys in response: {missing_keys}")

            logger.info(f"Classification successful: {result['code']}")
            return (
                result['code'],
                result['reasoning'],
                result['confidence']
            )

        except Exception as e:
            logger.error(f"Classification failed: {str(e)}", exc_info=True)
            return (
                "ERROR",
                f"Classification failed: {str(e)}",
                0.0
            )


def process_transactions(df: pd.DataFrame, chart_of_accounts: list) -> pd.DataFrame:
    """Process transactions in DataFrame and add classifications directly"""
    logger.info(f"Starting to process {len(df)} transactions")
    classifier = TransactionClassifier()

    # Create a fresh copy of the DataFrame
    df = df.copy()
    
    # Explicitly clear any previous results
    df['coa_agent'] = None
    df['coa_reason'] = None
    df['coa_confidence'] = None

    for i, (idx, row) in enumerate(df.iterrows()):
        print("row", row)
        logger.info(f"Processing transaction {idx}")
        transaction = row.to_dict()
        transaction['amount'] = transaction['amount']/100
        code, reasoning, confidence = classifier.classify_transaction(transaction, chart_of_accounts)
        
        # Update DataFrame using iloc instead of at
        df.iloc[i, df.columns.get_loc('coa_agent')] = code
        df.iloc[i, df.columns.get_loc('coa_reason')] = reasoning
        df.iloc[i, df.columns.get_loc('coa_confidence')] = confidence
        
        # Print summary of classification
        reason_preview = ' '.join(reasoning.split()[:10]) + '...' if reasoning else 'No reason provided'
        logger.info(f"Transaction {idx}:")
        logger.info(f"Account Code: {code}")
        logger.info(f"Reason Preview: {reason_preview}")
        logger.info("-" * 50)

    
    # Add verification logging
    processed_count = df[df['coa_agent'].notna()].shape[0]
    logger.info(f"Number of processed transactions: {processed_count}")
    
    logger.info("Finished processing all transactions")
    return df


""" Fetch and store chart of accounts into parsed_accounts """
import json

# Read the JSON file
with open('coa.json', 'r') as file:
    data = json.load(file)

# Extract only the required fields from each account
parsed_accounts = []
for account in data['Accounts']:
    parsed_account = {
        'code': account.get('AccountID', ''),
        'name': account.get('Name', ''),
        'status': account.get('Status', ''),
        'type': account.get('Type', ''),
        'taxtype': account.get('TaxType', ''),
        'description': account.get('Description', ''),  # Note: Not present in sample but included as requested
        'class': account.get('Class', ''),
        'reportingcode': account.get('ReportingCode', '')
    }
    parsed_accounts.append(parsed_account)

transactions = []  # Create empty list to store dictionaries

for index, row in df.iterrows():
    transaction = row.to_dict()
    transactions.append(transaction)

df_processed = process_transactions(df.copy(), parsed_accounts)

df_processed

2025-02-11 05:01:17,608 - INFO - Starting to process 37 transactions
2025-02-11 05:01:17,609 - INFO - Initializing TransactionClassifier
2025-02-11 05:01:17,610 - INFO - Processing transaction 2025-01-02 00:00:00
2025-02-11 05:01:17,610 - INFO - Processing transaction: {'creditorName': 'DVLA', 'remittanceInformationUnstructured': 'DVLA-LN61PYJ  000000000056504532 DDR', 'currency': 'GBP', 'amount': -13.12, 'coa_agent': None, 'coa_reason': None, 'coa_confidence': None}


row creditorName                                                         DVLA
remittanceInformationUnstructured    DVLA-LN61PYJ  000000000056504532 DDR
currency                                                              GBP
amount                                                              -1312
coa_agent                                                            None
coa_reason                                                           None
coa_confidence                                                       None
Name: 2025-01-02 00:00:00, dtype: object

=== Sending request to LLM ===


2025-02-11 05:01:19,991 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-11 05:01:20,010 - WARNING - Direct JSON parsing failed, attempting to extract JSON from response
2025-02-11 05:01:20,010 - INFO - Successfully extracted and parsed JSON from response
2025-02-11 05:01:20,010 - INFO - Classification successful: b04da64d-1580-4dd4-9de4-ac77130087d0
2025-02-11 05:01:20,011 - INFO - Transaction 2025-01-02 00:00:00:
2025-02-11 05:01:20,011 - INFO - Account Code: b04da64d-1580-4dd4-9de4-ac77130087d0
2025-02-11 05:01:20,011 - INFO - Reason Preview: The transaction is a payment made to the DVLA, which...
2025-02-11 05:01:20,011 - INFO - --------------------------------------------------
2025-02-11 05:01:20,012 - INFO - Processing transaction 2025-01-02 00:00:00
2025-02-11 05:01:20,012 - INFO - Processing transaction: {'creditorName': 'SUMUP *NEW LOOK BA', 'remittanceInformationUnstructured': 'SUMUP *NEW LOOK BA  ON 31 DEC BCC', 'currency': 'G


=== Raw LLM Response ===
```json
{
    "code": "b04da64d-1580-4dd4-9de4-ac77130087d0",
    "reasoning": "The transaction is a payment made to the DVLA, which is typically associated with vehicle-related expenses, such as registration fees or licenses. Therefore, it is best classified under 'Motor Vehicle Expenses', which covers costs related to the operation of business vehicles.",
    "confidence": 0.9
}
```
row creditorName                                        SUMUP *NEW LOOK BA
remittanceInformationUnstructured    SUMUP *NEW LOOK BA  ON 31 DEC BCC
currency                                                           GBP
amount                                                           -6000
coa_agent                                                         None
coa_reason                                                        None
coa_confidence                                                    None
Name: 2025-01-02 00:00:00, dtype: object

=== Sending request to LLM ===


2025-02-11 05:01:23,867 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-11 05:01:23,882 - WARNING - Direct JSON parsing failed, attempting to extract JSON from response
2025-02-11 05:01:23,883 - INFO - Successfully extracted and parsed JSON from response
2025-02-11 05:01:23,883 - INFO - Classification successful: f210f472-1c16-4826-b3e9-c2d1c7a8a757
2025-02-11 05:01:23,886 - INFO - Transaction 2025-01-02 00:00:00:
2025-02-11 05:01:23,886 - INFO - Account Code: f210f472-1c16-4826-b3e9-c2d1c7a8a757
2025-02-11 05:01:23,887 - INFO - Reason Preview: The transaction is a payment made through a point-of-sale system...
2025-02-11 05:01:23,887 - INFO - --------------------------------------------------
2025-02-11 05:01:23,888 - INFO - Processing transaction 2025-01-02 00:00:00
2025-02-11 05:01:23,888 - INFO - Processing transaction: {'creditorName': nan, 'remittanceInformationUnstructured': 'SAINSBURYS BANK Sainsburys Bank 31DEC 21.41 ATM', 'curr

...

2025-02-11 05:03:42,750 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-11 05:03:42,755 - WARNING - Direct JSON parsing failed, attempting to extract JSON from response
2025-02-11 05:03:42,755 - INFO - Successfully extracted and parsed JSON from response
2025-02-11 05:03:42,755 - INFO - Classification successful: 578df564-c408-4422-bc74-d8e1aabaf885
2025-02-11 05:03:42,756 - INFO - Transaction 2024-10-17 00:00:00:
2025-02-11 05:03:42,757 - INFO - Account Code: 578df564-c408-4422-bc74-d8e1aabaf885
2025-02-11 05:03:42,757 - INFO - Reason Preview: The transaction is a payment to Admiral for insurance based...
2025-02-11 05:03:42,757 - INFO - --------------------------------------------------
2025-02-11 05:03:42,758 - INFO - Processing transaction 2024-10-09 00:00:00
2025-02-11 05:03:42,758 - INFO - Processing transaction: {'creditorName': 'Vodafone', 'remittanceInformationUnstructured': 'VODAFONE LTD  7082364969-1001 DDR', 'currency': 'GBP'


=== Raw LLM Response ===
```json
{
    "code": "578df564-c408-4422-bc74-d8e1aabaf885",
    "reasoning": "The transaction is a payment to Admiral for insurance based on the remittance information 'ADMIRAL INSURANCE'. In the chart of accounts, 'Insurance' is marked as an overhead expense for costs related to insuring business assets, which directly aligns with this type of transaction.",
    "confidence": 0.95
}
```
row creditorName                                                  Vodafone
remittanceInformationUnstructured    VODAFONE LTD  7082364969-1001 DDR
currency                                                           GBP
amount                                                           -1295
coa_agent                                                         None
coa_reason                                                        None
coa_confidence                                                    None
Name: 2024-10-09 00:00:00, dtype: object

=== Sending request to LLM ===


2025-02-11 05:03:45,506 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-11 05:03:45,513 - WARNING - Direct JSON parsing failed, attempting to extract JSON from response
2025-02-11 05:03:45,514 - INFO - Successfully extracted and parsed JSON from response
2025-02-11 05:03:45,515 - INFO - Classification successful: ab941bc6-8dee-48d9-8ac1-b678b247648b
2025-02-11 05:03:45,515 - INFO - Transaction 2024-10-09 00:00:00:
2025-02-11 05:03:45,516 - INFO - Account Code: ab941bc6-8dee-48d9-8ac1-b678b247648b
2025-02-11 05:03:45,516 - INFO - Reason Preview: The transaction description 'VODAFONE LTD' and the creditor name 'Vodafone'...
2025-02-11 05:03:45,516 - INFO - --------------------------------------------------
2025-02-11 05:03:45,517 - INFO - Number of processed transactions: 37
2025-02-11 05:03:45,517 - INFO - Finished processing all transactions



=== Raw LLM Response ===
```json
{
    "code": "ab941bc6-8dee-48d9-8ac1-b678b247648b",
    "reasoning": "The transaction description 'VODAFONE LTD' and the creditor name 'Vodafone' suggests that this is a utility bill payment for services like telephone or internet. This matches the 'Telephone & Internet' account, as these are typical services Vodafone provides.",
    "confidence": 0.95
}
```


,creditorName,remittanceInformationUnstructured,currency,amount,coa_agent,coa_reason,coa_confidence
bookingDate,,,,,,,
2025-01-02,DVLA,DVLA-LN61PYJ 000000000056504532 DDR,GBP,-1312,b04da64d-1580-4dd4-9de4-ac77130087d0,"The transaction is a payment made to the DVLA,...",0.9
2025-01-02,SUMUP *NEW LOOK BA,SUMUP *NEW LOOK BA ON 31 DEC BCC,GBP,-6000,f210f472-1c16-4826-b3e9-c2d1c7a8a757,The transaction is a payment made through a po...,0.7
2025-01-02,NaN,SAINSBURYS BANK Sainsburys Bank 31DEC 21.41 ATM,GBP,-5000,f210f472-1c16-4826-b3e9-c2d1c7a8a757,"The transaction description mentions 'ATM', in...",0.8
2024-12-27,KLARNA*ZOOM COMMUN,KLARNA*ZOOM COMMUN ON 26 DEC BCC,GBP,-1559,f8ab6764-b60f-4ef7-b5d4-1406fecb2cf6,The transaction appears to be a payment to Zoo...,0.95
2024-12-27,THE HUB DENTAL PRA,THE HUB DENTAL PRA ON 24 DEC BCC,GBP,-5000,c897be00-cd56-426f-9c51-3dc4a5bb7442,The transaction suggests a payment to 'THE HUB...,0.7
2024-12-24,AMERICAN EXP 3773 PB166643916980508,AMERICAN EXP 3773 PB166643916980508 FT,GBP,-599,f210f472-1c16-4826-b3e9-c2d1c7a8a757,The transaction is a small charge from 'AMERIC...,0.9
2024-12-24,AMERICAN EXP 3773 PB313981532278988,AMERICAN EXP 3773 PB313981532278988 FT,GBP,-21320,f210f472-1c16-4826-b3e9-c2d1c7a8a757,The transaction appears to be a payment made t...,0.9
2024-12-24,CRESCENT ADVISORS MICHAEL ALI DLA,CRESCENT ADVISORS MICHAEL ALI DLA FT,GBP,-30000,60c328b1-2c88-4513-83aa-206ae420b2c1,The transaction is a payment to 'CRESCENT ADVI...,0.9
2024-12-24,Barclaycard,MR MICHAEL ALI 4929158426786006 BBP,GBP,-14538,422059c7-2c27-46b6-bf66-cacdf7c08902,The transaction appears to be a payment toward...,0.95


## To evaluate reponses of LLM

In [6]:
df_processed.to_csv("transactions_ai_reconciled.csv")